### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
!pip install sqlalchemy==1.3.9

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
!pip install -q pandas==1.1.5

In [ ]:
%sql sqlite:///my_data1.db

In [ ]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

**Note:This below code is added to remove blank rows from table**


In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

In [ ]:
%sql SELECT * FROM SPACEXTABLE

## EXTRA QUERIES

### QUERY 1

##### Find the launch site with the highest success rate for missions:


In [ ]:
%sql SELECT Launch_Site, 
       COUNT(CASE WHEN Mission_Outcome = 'Success' THEN 1 ELSE NULL END) AS Successful_Missions,
       COUNT(*) AS Total_Missions,
       (COUNT(CASE WHEN Mission_Outcome = 'Success' THEN 1 ELSE NULL END) * 100.0 / COUNT(*)) AS Success_Rate
    FROM SPACEXTABLE
    GROUP BY Launch_Site
    ORDER BY Success_Rate DESC
    LIMIT 1;


### QUERY 2


#####  List the customers who had successful missions and the number of successful missions for each customer: 


In [ ]:
%sql SELECT Customer, COUNT(*) AS Successful_Missions
    FROM SPACEXTABLE
    WHERE Mission_Outcome = 'Success'
    GROUP BY Customer;

### QUERY 3




##### Find the booster version that had the highest total payload mass, along with the corresponding launch site:


In [ ]:
%sql SELECT Booster_Version, Launch_Site
    FROM SPACEXTABLE
    WHERE (Booster_Version, PAYLOAD_MASS__KG_) IN (
        SELECT Booster_Version, MAX(PAYLOAD_MASS__KG_)
        FROM SPACEXTABLE
        GROUP BY Booster_Version
    );

### QUERY 4




##### List the payloads for missions with a successful landing outcome and a payload mass greater than the average payload mass for all missions:


In [ ]:
%sql SELECT Payload
    FROM SPACEXTABLE
    WHERE Landing_Outcome = 'Success'
    AND PAYLOAD_MASS__KG_ > (
        SELECT AVG(PAYLOAD_MASS__KG_)
        FROM SPACEXTABLE
    );

### QUERY 5

##### Retrieve the launch site with the highest success rate for missions with payloads to a specific orbit:



In [ ]:
%sql SELECT Launch_Site
    FROM SPACEXTABLE
    WHERE Orbit = 'Desired_Orbit' AND Mission_Outcome = 'Success'
    GROUP BY Launch_Site
    HAVING COUNT(*) = (
        SELECT MAX(Count_Mission)
        FROM (
            SELECT Launch_Site, COUNT(*) AS Count_Mission
            FROM SPACEXTABLE
            WHERE Orbit = 'Desired_Orbit'
            GROUP BY Launch_Site, Mission_Outcome
        )
    );

### QUERY 6

##### Find the top 5 customers with the highest total payload mass and the corresponding missions:


In [ ]:
%sql SELECT Customer, Payload, Total_Payload_Mass
    FROM (
        SELECT Customer, Payload, SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
        FROM SPACEXTABLE
        GROUP BY Customer, Payload
        ORDER BY Total_Payload_Mass DESC
    ) AS RankedPayloads
    LIMIT 5;

### QUERY 7




##### Calculate the average payload mass for missions with a successful landing outcome and compare it to the average payload mass for missions with a failed landing outcome for each booster version:


In [ ]:
%sql SELECT Booster_Version, AVG(PAYLOAD_MASS__KG_) AS Avg_Successful_Landing_Mass, Avg_Failed_Landing_Mass
    FROM (
        SELECT Booster_Version, AVG(CASE WHEN Landing_Outcome = 'Success' THEN PAYLOAD_MASS__KG_ ELSE NULL END) AS Avg_Successful_Landing_Mass
        FROM SPACEXTABLE
        GROUP BY Booster_Version
    ) AS SuccessfulLanding
    LEFT JOIN (
        SELECT Booster_Version, AVG(CASE WHEN Landing_Outcome = 'Failure' THEN PAYLOAD_MASS__KG_ ELSE NULL END) AS Avg_Failed_Landing_Mass
        FROM SPACEXTABLE
        GROUP BY Booster_Version
    ) AS FailedLanding ON SuccessfulLanding.Booster_Version = FailedLanding.Booster_Version;